In [ ]:
%pip install -Uq -r requirements.txt 2> /dev/null && echo "All required packages are installed."

## Lab 3: Deploy complete solution on AgentCore

### Overview
Lab 2 covered packaging and deploying remote tools so agents can call them. Lab 3 builds on that by packaging entire agents, configuring identity so agents can obtain authentication tokens securely, and launching agent runtime sessions in a managed environment. In this lab you will deploy Strands Agents to AgentCore Runtime, configure AgentCore Identity for secure credential management, and explore observability features to monitor agent behavior. You'll package an agent, create credential providers, and launch an agent runtime session.

### Goals
- Package and deploy an agent to AgentCore Runtime (using `bedrock_agentcore_starter_toolkit`).
- Configure AgentCore Identity credential providers for secure access to tools and resources.
- Invoke an agent runtime session and inspect streamed or batched responses.
- Use AgentCore observability tools (CloudWatch) to review agent traces and logs.

### Key tools & libraries covered
- [AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) - A managed service that hosts and runs the agent. The runtime provides a secure and scalable environment with user session isolation, resource management, and monitoring capabilities.
- [AgentCore Identity](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity.html) - A service that enables the agent to assume a secure identity when deployed to a runtime. This identity is used to authenticate and authorize the agent's access to tools, resources, and data.
- [bedrock_agentcore_starter_toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit) to simplify building, packaging, and deploying tools
- [AgentCore Control Plane API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agentcore-control.html) used to deploy AgentCore resources such as agents and tools programmatically
- [AgentCore DataPlane API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agentcore.html) used to invoke AgentCore resources programmatically
- [AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) - A set of tools and services that provide insights into the agent's performance, behavior, and interactions. Observability helps in debugging, optimizing, and auditing the agent's operations.

In [1]:
import boto3
from pathlib import Path
import json

from rich import print as rprint
from rich.markdown import Markdown

from bedrock_agentcore_starter_toolkit.operations.runtime import (
    configure_bedrock_agentcore,
    launch_bedrock_agentcore,
)

from bedrock_agentcore.services.identity import IdentityClient

In [2]:
REGION = boto3.Session().region_name
SECRETS_MANAGER_CLIENT = boto3.client("secretsmanager")
AGENT_RUNTIME_CLIENT = boto3.client('bedrock-agentcore', region_name=REGION)

# this json contains various workshop resource ARNs that were created prior to the workshop
WORKSHOP_RESOURCES = json.loads(Path("workshop_resources.json").read_text())

# this json contains various lab resource IDs that were created during the workshop
LAB_RESOURCES = json.loads(Path("lab_resources.json").read_text())

IDENTITY_CLIENT = IdentityClient(region=REGION) # used to invoke AgentCore Identity APIs

### AgentCore Identity
The remote tools deployed in the prior notebook were configured with OAuth2 based authentication and in our example code we manually obtained the access tokens and passed them to the tool clients. In a production deployment, this approach is not scalable or secure. Instead, we can leverage AgentCore Identity to manage the agent's identity and dynamically obtain access tokens when needed.

The first step in configuring this is creating a [credential provider](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/resource-providers.html) which will use a combination of the user's identity and the agents's identity to request access tokens for the tools it needs to access.

In [3]:
credential_provider_name = "kyc-mcp-credential-provider"

# check if the credential provider already exists
existing_credential_providers = [
    provider
    for provider in IDENTITY_CLIENT.identity_client.list_oauth2_credential_providers()["credentialProviders"]
    if provider["name"] == credential_provider_name
]

if existing_credential_providers:
    print(f"Credential provider {credential_provider_name} already exists")
    cognito_credential_provider = existing_credential_providers[0]
    
else:
    # create the credential provider if it does not exist
    client_secret = SECRETS_MANAGER_CLIENT.get_secret_value(
        SecretId=WORKSHOP_RESOURCES["CognitoClientSecretArn"]
    )["SecretString"]

    cognito_credential_provider = IDENTITY_CLIENT.create_oauth2_credential_provider(
        {
            "name": credential_provider_name,
            "credentialProviderVendor": "CustomOauth2",
            "oauth2ProviderConfigInput": {
                "customOauth2ProviderConfig": {
                    "clientId": WORKSHOP_RESOURCES["CognitoClientId"],
                    "clientSecret": client_secret,
                    "oauthDiscovery": {
                        "authorizationServerMetadata": {
                            "issuer": WORKSHOP_RESOURCES["CognitoIssuerUrl"],
                            "authorizationEndpoint": WORKSHOP_RESOURCES[
                                "CognitoAuthEndpoint"
                            ],
                            "tokenEndpoint": WORKSHOP_RESOURCES["CognitoTokenEndpoint"],
                            "responseTypes": ["code", "token"],
                        }
                    },
                }
            },
        }
    )

### AgentCore Runtime
With the credential provider created, we can now move on to deploying the agent itself using AgentCore Runtime. The entrypoint code for the agent can be found in [workspace/kyc_agent/kyc_agent.py](workspace/kyc_agent/kyc_agent.py). The code is similar to what we had in the prior notebook, with a few key differences:

1. The agent invocation function is decorated with `@APP.entrypoint`, Here `APP` is an instance of `BedrockAgentCoreApp` class provided by the [AgentCore Python SDK](https://github.com/aws/bedrock-agentcore-sdk-python). This decorator marks the function as the main entrypoint for the agent when it is deployed to AgentCore Runtime.
2. The entrypoint function is also decorated with `@requires_access_token` which will leverage the credential provider we created earlier to obtain OAuth2 access tokens for the tools the agent needs to access. Behind the scenes, it will use a combination of the user's unique identity and the agent's identity to request access tokens from the credential provider.

The AgentCore Python SDK provides convenience features which reduce boilerplate code. However using it is not a requirement as AgentCore runtime primarily requires you to create a Docker image that exposes a service on port 8080 which adheres to the [AgentCore Runtime HTTP Contract](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-http-protocol-contract.html).

Note in the below agent configuration we do not specify `authorizer_configuration` which means that IAM based authentication will be used by default for inbound access control. Alternatively, just as we did in the prior notebook with the KYC MCP tools, we could have also configured OAuth2 based authentication for the agent itself. The difference is with IAM based authentication, we will have to pass a unique `user_id` for each user session when invoking the agent. This `user_id` will be used by AgentCore Identity to uniquely identify the user and issue access tokens accordingly, with Oauth2 based authentication, AgentCore Identity would obtain the user's identity from the JWT token presented during authentication.

In [4]:
kyc_agent_configuration = configure_bedrock_agentcore(
    agent_name="kyc_agent",
    protocol="HTTP",                                                # MCP and A2A protocols are also supported
    execution_role=WORKSHOP_RESOURCES["AgentCoreExecutionRole"],    # this role will be assumed by the runtime instance
    source_path="workspace/kyc_agent",                              # files in this directory will be packaged into a Docker image
    entrypoint_path=Path("workspace/kyc_agent/kyc_agent.py"),       # this script will be executed when the container starts
    non_interactive=True,
    code_build_execution_role=WORKSHOP_RESOURCES["CodeBuildRole"],   # role for building the Docker image
    memory_mode="NO_MEMORY",                                         # Memory has already been created in Lab 1
    verbose=True,
)

Verbose mode enabled
Configuring BedrockAgentCore agent: kyc_agent
Build directory: /home/sagemaker-user/strands-agentcore-workshop
Source path: workspace/kyc_agent
Bedrock AgentCore name: kyc_agent
Entrypoint path: workspace/kyc_agent/kyc_agent.py
Retrieving AWS account information...
AWS account ID: 780660052610
AWS region: us-west-2
Initializing container runtime with: default
Using execution role: arn:aws:iam::780660052610:role/BedrockAgentCoreRole
Memory disabled
Unable to read existing memory configuration: Agent 'kyc_agent' not found. Available agents: ['kyc_mcp_tooling']
Using separate CodeBuild execution role: arn:aws:iam::780660052610:role/CodeBuildExecutionRole
Attempting to find Bedrock AgentCore instance name in workspace/kyc_agent/kyc_agent.py
Generating Dockerfile with parameters:
  Entrypoint: workspace/kyc_agent/kyc_agent.py
  Build directory: /home/sagemaker-user/strands-agentcore-workshop
  Bedrock AgentCore name: bedrock_agentcore
  Region: us-west-2
  Enable observ

⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: .bedrock_agentcore/kyc_agent/Dockerfile
Agent name from BedrockAgentCoreApp: kyc_agent
Config path: /home/sagemaker-user/strands-agentcore-workshop/.bedrock_agentcore.yaml
Using entrypoint format: workspace/kyc_agent/kyc_agent.py
ECR auto-create: True
Creating BedrockAgentCoreConfigSchema with following parameters:
  Name: kyc_agent
  Entrypoint: workspace/kyc_agent/kyc_agent.py
  Platform: linux/arm64
  Container runtime: docker
  Execution role: arn:aws:iam::780660052610:role/BedrockAgentCoreRole
  ECR repository: Auto-create
  Enable observability: True
  Request header configuration: None
Changing default agent from 'kyc_mcp_tooling' to 'kyc_agent'
Configuration saved with agent: kyc_agent


In [5]:
launch_agent_server = launch_bedrock_agentcore(
    config_path=kyc_agent_configuration.config_path,
    # environment variables can be used to pass configuration to the agent at runtime
    env_vars={"OAUTH2_ID_PROVIDER": credential_provider_name,
              "MCP_URL": LAB_RESOURCES["KYC_MCP_Runtime_Url"],
              "MODEL_ID": "global.anthropic.claude-haiku-4-5-20251001-v1:0"},
    auto_update_on_conflict=True
)

Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'kyc_agent' to account 780660052610 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: kyc_agent
✅ ECR repository available: 780660052610.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-kyc_agent
Using execution role from config: arn:aws:iam::780660052610:role/BedrockAgentCoreRole
Preparing CodeBuild project and uploading source...


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-kyc_agent


Using CodeBuild role from config: arn:aws:iam::780660052610:role/CodeBuildExecutionRole
Using dockerignore.template with 45 patterns for zip filtering
Including Dockerfile from /home/sagemaker-user/strands-agentcore-workshop/.bedrock_agentcore/kyc_agent in source.zip
Uploaded source to S3: kyc_agent/source.zip
Created CodeBuild project: bedrock-agentcore-kyc_agent-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 9.3s
🔄 DOWNLOAD_SOURCE started (total: 10s)
✅ DOWNLOAD_SOURCE completed in 1.0s
🔄 INSTALL started (total: 11s)
✅ INSTALL completed in 1.1s
🔄 BUILD started (total: 12s)
✅ BUILD completed in 23.8s
🔄 POST_BUILD started (total: 36s)
✅ POST_BUILD completed in 21.7s
🔄 COMPLETED started (total: 58s)
✅ COMPLETED completed in 1.0s
🎉 CodeBuild completed successfully in 0m 58s
CodeBuild completed successfully
✅ CodeBuild

When invoking the agent, we have to provide a unique `runtimeUserId` as well as a `runtimeSessionId`. Each user session is isolated from others, so multiple users can interact with the same agent concurrently without interference. The sessions are shortlived though and will expire after a period of inactivity (default is 15 minutes). However with AgentCore Memory, the session state can be persisted and restored across sessions if needed.

In [7]:

response = AGENT_RUNTIME_CLIENT.invoke_agent_runtime(
    agentRuntimeArn=launch_agent_server.agent_arn,
    runtimeSessionId='1234567890123456789012345678901671', # unique session id for tracking and isolation
    runtimeUserId='test-user-123',                         # unique user id
    payload=json.dumps({"input": "Gather information about applicant William Mcgee"}) # the input prompt to the agent
)

AgentCore Runtime provides a wide array of invocation options including synchronous invocation with [streamed responses](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/response-streaming.html), and [asynchronous invocations](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-long-run.html) that can run for up to 8 hours. 

In the deployed example, the agent was configured to provide streamed responses, so we can iterate over the response generator to receive partial outputs as they are produced by the agent. This is particularly useful for agents that may take a long time to complete their tasks, as it allows users to see progress in real-time.

In [8]:
if "text/event-stream" in response.get("contentType", ""):
    content = []
    for line in response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[7:-1]
                line = line.encode().decode('unicode_escape')
                print(line, end="")
                content.append(line)
else:
    try:
        events = []
        for event in response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]

"error": "the client session is currently running", "error_type": "MCPClientInitializationError", "message": "An error occurred during streaming"

### AgentCore Observability
With the agent deployed on a remote runtime, we can no longer see the logs and traces directly in our local environment. Even if we could, it would not be practical to do so at scale when multiple users are interacting with the agent concurrently. Instead, we can leverage AgentCore Observability features to monitor and debug our agents in a scalable manner. AgentCore Observability provides a number of features including:
- **CloudWatch Logs**: All logs generated by the agent and its tools are automatically sent to CloudWatch Logs. This allows us to view and search logs in a centralized location, making it easier to debug issues and monitor agent behavior.
- **Agent Level Metrics**: AgentCore Runtime automatically collects and publishes metrics about the agent's performance and behavior to CloudWatch Metrics. This includes metrics such as request latency, error rates, and resource utilization.
- **Session Level Metrics**: In addition to agent level metrics, AgentCore Runtime also collects and publishes session level metrics providing insights into individual user sessions.
- **Tracing**: AgentCore Runtime integrates with AWS X-Ray to provide distributed tracing capabilities giving insights into the agent's interactions with its tools and resources. This helps in identifying bottlenecks and optimizing performance.

Run the cell below to generate a direct link to the GenAI Observability dashboard in CloudWatch where you can explore logs, metrics, and traces for the deployed agent.

In [9]:
observability_url = f"https://{REGION}.console.aws.amazon.com/cloudwatch/home?region={REGION}#gen-ai-observability/agent-core/agent-alias/{launch_agent_server.agent_id}/endpoint/DEFAULT/agent/kyc_agent"
rprint(Markdown(f"**You can click [here]({observability_url}) to view the observability dashboard.**"))

You can click ]8;id=919567;https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#gen-ai-observability/agent-core/agent-alias/kyc_agent-yyXvTfClvV/endpoint/DEFAULT/agent/kyc_agent\here]8;;\ to view the observability dashboard.

In [10]:
try:
    with open("lab_resources.json", "r") as f:
        data = json.load(f)
except FileNotFoundError:
    data = {}

# Add or update the keys
data.update({
    "agentRuntimeId": launch_agent_server.agent_id,
    "credentialProviderName": credential_provider_name,
    
})

# Write the updated data back to the file
with open("lab_resources.json", "w") as f:
    json.dump(data, f, indent=4)

### Conclusion
In this lab you learned how to deploy a complete agent solution using AgentCore Runtime and AgentCore Identity. You packaged an agent, configured secure credential management, launched an agent runtime session, and explored observability features to monitor agent behavior. With these skills, you can now build and deploy scalable, secure AI agents using AgentCore services.